## Result 4: MOD1/2 specific markers are expressed over large region

### Data

In [ ]:
from scipy.io import mmread
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

#### signals in tissue section

In [ ]:
signal_data_folder_path = Path("../data/mouse_hypothalamus/raw/")

In [ ]:
columns = [
    "Centroid_X",
    "Centroid_Y",
    "Centroid_Z",
    "Gene_name",
    "Cell_name",
    "Total_brightness",
    "Area",
    "Error_bit",
    "Error_direction",
]

signal_coordinate_df = pd.read_csv(
    signal_data_folder_path / "merfish_barcodes_example.csv", usecols=columns
).rename(
    columns={
        "Centroid_X": "x",
        "Centroid_Y": "y",
        "Centroid_Z": "z",
        "Gene_name": "gene",
    }
)


# remove dummy molecules
signal_coordinate_df = signal_coordinate_df.loc[
    ~signal_coordinate_df["gene"].str.contains("Blank|NegControl"),
]

signal_coordinate_df["gene"] = signal_coordinate_df["gene"].astype("category")

# shift the coordinates to avoid the negative values
coordinate_x_m =  signal_coordinate_df['x'].min()
coordinate_y_m =  signal_coordinate_df['y'].min()
signal_coordinate_df['x'] = signal_coordinate_df['x'] - coordinate_x_m
signal_coordinate_df['y'] = signal_coordinate_df['y'] - coordinate_y_m

# make a copy to avoid SettingWithCopyWarning
signal_coordinate_df = signal_coordinate_df.copy()

#### Results of BANKSY

In [ ]:
banksy_folder_path = Path("../data/banksy_results/")

In [ ]:
columns = [
    "Centroid_X",
    "Centroid_Y",
    "Bregma",
    "lam0.2",
]

banksy_result = pd.read_csv(
    banksy_folder_path / 'banksy_cluster.txt', usecols=columns, sep = '\t'
).rename(
    columns={
        "Centroid_X": "x",
        "Centroid_Y": "y",
        "Bregma": "Bregma",
        "lam0.2": "banksy_cluster",
    }
)

banksy_result = banksy_result[banksy_result['Bregma'] == -0.24]

banksy_result['x'] = banksy_result['x'] - coordinate_x_m
banksy_result['y'] = banksy_result['y'] - coordinate_y_m

banksy_result = banksy_result.copy()

#### MOD
BANKSY cluster:  
- 7: MOD_gm  
- 8: MOD_wm

In [ ]:
OD_result = banksy_result[(banksy_result['banksy_cluster']==7)|(banksy_result['banksy_cluster']==8)]
OD_wm_df = banksy_result[banksy_result['banksy_cluster'] == 8]
OD_gm_df = banksy_result[banksy_result['banksy_cluster'] == 7]

#### Marker Genes

differentially expressed genes identified by BANKSY

In [ ]:
# all differentially expressed genes
DE_genes = ['Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3', 'Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck']
# DE_genes_gm: 7
DE_genes_gm = ['Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3']
# DE_genes_wm: 8
DE_genes_wm = ['Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck']

In [ ]:
# Marker Genes Signals
DE_gene_signals = signal_coordinate_df[signal_coordinate_df['gene'].isin(DE_genes)].copy()
OD_wm_signals = signal_coordinate_df[signal_coordinate_df['gene'].isin(DE_genes_wm)].copy()
OD_gm_signals = signal_coordinate_df[signal_coordinate_df['gene'].isin(DE_genes_gm)].copy()

### MOD1

marker genes: Lpar1

In [ ]:
import matplotlib.lines as mlines
from matplotlib.colors import Normalize

plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    OD_wm_signals["x"],
    OD_wm_signals["y"],
    s=1,
    c=plt.cm.Oranges(norm(OD_wm_signals['Total_brightness'])), 
    alpha=0.8,
    # label='OD_wm DE genes Signals'
)

for i in range(len(OD_wm_df)):
    plt.text(
        OD_wm_df["x"].iloc[i], 
        OD_wm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='red',
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='red', marker='x', markersize=10, label="MOD1 centroid", ls='')

# plt.legend(handles=[centroid_legend], loc='upper right', fontsize=17)

plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("MOD1 Markers", fontsize=17)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
import matplotlib.patches as patches

plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    OD_wm_signals["x"],
    OD_wm_signals["y"],
    s=1,
    c=plt.cm.Oranges(norm(OD_wm_signals['Total_brightness'])), 
    alpha=0.8,
    label='Lpar1'
)

for i in range(len(OD_wm_df)):
    plt.text(
        OD_wm_df["x"].iloc[i], 
        OD_wm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='red',
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='red', marker='x', markersize=7, label="MOD1 centroid", ls='')
# scatter_proxy = mlines.Line2D([], [], color=plt.cm.Oranges(0.7), marker='o', markersize=7, linestyle='None', label="Lpar1")
# plt.legend(handles=[scatter_proxy, centroid_legend], loc='lower right', fontsize=17)

# plt.legend(handles=[centroid_legend], loc='lower right', fontsize=17)

# Define the rectangle position and size
x_rect, y_rect = 750, 100  # Bottom-left corner of the rectangle
width, height = 220, 1400  # Width and height of the rectangle

# Get the current axis
ax = plt.gca()

# Create a rectangle patch
rect = patches.Rectangle(
    (x_rect, y_rect), width, height,
    linewidth=1, edgecolor='red', facecolor='none'
)

# Add the rectangle to the plot
ax.add_patch(rect)


plt.xlabel("(μm)")
plt.ylabel("(μm)")
# plt.xticks([])
# plt.yticks([])
plt.title("Lpar1", fontsize=17)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

### MOD2

In [ ]:
# import matplotlib.lines as mlines

plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    OD_gm_signals["x"],
    OD_gm_signals["y"],
    s=1,
    c=plt.cm.Blues(norm(OD_gm_signals['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='blue', marker='x', markersize=10, label="MOD2 centroid", ls='')

# plt.legend(handles=[centroid_legend], loc='upper right', fontsize=17)

plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("MOD2 Markers", fontsize=17)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Cbln2_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Cbln2'])].copy()

plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Cbln2_signal["x"],
    Cbln2_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Cbln2_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='blue', marker='x', markersize=8, label="MOD2 centroid", ls='')
# scatter_proxy = mlines.Line2D([], [], color=plt.cm.Blues(0.7), marker='o', markersize=8, linestyle='None', label="Cbln2")
# plt.legend(handles=[scatter_proxy, centroid_legend], loc='lower right', fontsize=17)

plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Cbln2", fontsize=17)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Dgkk_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Dgkk'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Dgkk_signal["x"],
    Dgkk_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Dgkk_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='red', marker='x', markersize=8, label="MOD2 centroid", ls='')
# scatter_proxy = mlines.Line2D([], [], color=plt.cm.Oranges(0.7), marker='o', markersize=8, linestyle='None', label="Dgkk")
# plt.legend(handles=[scatter_proxy, centroid_legend], loc='lower right', fontsize=17)

plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Dgkk")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Gad1_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Gad1'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Gad1_signal["x"],
    Gad1_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Gad1_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)

# centroid_legend = mlines.Line2D([], [], color='red', marker='x', markersize=8, label="MOD2 centroid", ls='')
# scatter_proxy = mlines.Line2D([], [], color=plt.cm.Oranges(0.7), marker='o', markersize=8, linestyle='None', label="Gad1")
# plt.legend(handles=[scatter_proxy, centroid_legend], loc='lower right', fontsize=17)

plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Gad1")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Gjc3_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Gjc3'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Gjc3_signal["x"],
    Gjc3_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Gjc3_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Gjc3")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Mlc1_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Mlc1'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Mlc1_signal["x"],
    Mlc1_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Mlc1_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Mlc1")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Plin3_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Plin3'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Plin3_signal["x"],
    Plin3_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Plin3_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Plin3")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()


In [ ]:
Syt4_signal = signal_coordinate_df[signal_coordinate_df['gene'].isin(['Syt4'])].copy()
plt.figure(figsize=(10, 10), dpi=600)

norm = Normalize(0, 2000)

scatter_gm = plt.scatter(
    Syt4_signal["x"],
    Syt4_signal["y"],
    s=1,
    c=plt.cm.Blues(norm(Syt4_signal['Total_brightness'])), 
    alpha=0.8,
    # label='OD_gm DE genes Signals'
)

for i in range(len(OD_gm_df)):
    plt.text(
        OD_gm_df["x"].iloc[i], 
        OD_gm_df["y"].iloc[i], 
        "x", 
        fontsize=7, 
        color='blue', 
        ha='center', 
        va='center'
    )

x_min, x_max = signal_coordinate_df["x"].min(), signal_coordinate_df["x"].max()
y_min, y_max = signal_coordinate_df["y"].min(), signal_coordinate_df["y"].max()
plt.xlim(x_min - 10, x_max + 10)
plt.ylim(y_min - 10, y_max + 10)
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.xticks([])
plt.yticks([])
plt.title("Syt4")

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()